# Analisis de Egresos Hospitalarios

In [4]:
import importlib

import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import lectura_archivos as fr
import calculo_metricas as aux

from functools import reduce

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

importlib.reload(fr)
importlib.reload(aux)


<module 'calculo_metricas' from 'c:\\Users\\ppizarro\\Desktop\\Javier Rojas\\Javier\\trabajo\\SSMO\\EPH\\modulo_gestion\\1.analisis_problemas_de_salud\\egresos-hospitalarios\\calculo_metricas.py'>

In [5]:
HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax


## Lectura de archivos

- Se leen todos los archivos que estan en input/utf-8. Los archivos estan separados por ;
- Se dejan solamente los egresos de los diagnosticos que **SI** tenga el Tórax
- Se cambia la codificacion de INTERV_Q de 2 (No, 1 es SI) a 0.
- Se cambia la codificacion de CONDICION_EGRESO (orig 1 = Vivo, 2 = Fallecido) de 1 -> 0 y 2 -> 1
- Se cambia la codificacion de GLOSA_REGION_RESIDENCIA {
                        "Del Libertador B. O'Higgins": "del Libertador General Bernardo O'Higgins",
                        "De Aisén del Gral. C. Ibáñez del Campo": "Aysén del General Carlos Ibáñez del Campo",
                    }

In [6]:
df = fr.leer_archivos(HOSPITAL_A_ANALIZAR)

# Analisis de Egresos Hospitalarios

Se calculan las siguientes metricas:

- Número de Egresos (n_egresos)
- Días de Estada promedio (dias_estada_promedio)
- Cantidad de Intervenciones Quirúrgicas (n_int_q)
- Cantidad de Muertes (n_muertos)

Todas estas metricas estan calculadas a nivel de Año, Establecimiento de Salud y Diagnóstico. Por
ejemplo:

***"En el año 2019, el Hospital San José (código XXXXXXX) tuvo XX egresos, con XX dias estada_promedio,
con XX intervenciones_quirúrgicas y XX muertos para el diagnóstico A24.4."***

Este análisis se realizó entre 2001 y 2020 para todos los hospitales/clínicas de Chile.

In [7]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = aux.obtener_metricas_egresos(df, AGRUPACION).collect(streaming=True)


# Analisis de Estratos

In [8]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]
unir_tablas_por = AGRUPACION

In [9]:
dict_estratos = aux.obtener_diccionario_estratos(df, HOSPITAL_A_ANALIZAR)

In [10]:
dict_resultados_estratos = aux.obtener_resumen_por_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

resultado = reduce(aux.left_join_consecutivo, dict_resultados_estratos.values())

# Analisis Sociodemográfico

In [34]:
desglose_sociodemografico = [
    "ANO_EGRESO",
    "REGION_RESIDENCIA",
    "GLOSA_REGION_RESIDENCIA",
    "COMUNA_RESIDENCIA",
    "GLOSA_COMUNA_RESIDENCIA",
    "DIAG1"
]

schema_optimo = {'ANO_EGRESO': pl.Int32,
 'REGION_RESIDENCIA': pl.Categorical,
 'GLOSA_REGION_RESIDENCIA': pl.Categorical,
 'COMUNA_RESIDENCIA': pl.Categorical,
 'GLOSA_COMUNA_RESIDENCIA': pl.Categorical,
 'DIAG1': pl.Utf8}

df_socio = (
    pl.scan_csv("input/utf-8/*.csv", separator=";", dtypes=schema_optimo)
    .filter(pl.col("ESTABLECIMIENTO_SALUD") == 112103)
    .groupby(desglose_sociodemografico)
    .agg(pl.col("DIAG1").count().alias('conteo'))
)

In [37]:
with pl.StringCache():
    resultado_sociodemo = df_socio.collect(streaming=True)

In [38]:
resultado_sociodemo

ANO_EGRESO,REGION_RESIDENCIA,GLOSA_REGION_RESIDENCIA,COMUNA_RESIDENCIA,GLOSA_COMUNA_RESIDENCIA,DIAG1,conteo
i32,cat,cat,cat,cat,str,u32
2002,"""13""","""Metropolitana …","""13122""","""Peñalolén""","""J398""",4
2004,"""13""","""Metropolitana …","""13108""","""Independencia""","""I679""",1
2004,"""13""","""Metropolitana …","""13126""","""Quinta Normal""","""J931""",2
2006,"""13""","""Metropolitana …","""13605""","""Peñaflor""","""J47X""",1
2009,"""15""","""De Arica y Par…","""15101""","""Arica""","""Q213""",2
2010,"""04""","""De Coquimbo""","""4102""","""Coquimbo""","""A182""",1
2011,"""13""","""Metropolitana …","""13131""","""San Ramón""","""J90X""",2
2015,"""13""","""Metropolitana …","""13106""","""Estación Centr…","""J841""",2
2015,"""05""","""De Valparaíso""","""5201""","""Isla de Pascu…","""I495""",1
